In [1]:
### react agent use stock query API, and tavily search tool and langchain calculators from packages
import os
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

#https://python.langchain.com/docs/integrations/tools/tavily_search/

In [2]:
llm2 = ChatOpenAI(
    api_key= API_KEY,
    model="gpt-4",
    temperature=0
)

C:\Users\difen\AppData\Local\Temp\ipykernel_21560\1258198397.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm2 = ChatOpenAI(


In [ ]:
### test API case use my function, not depdent on lainchain library
import yfinance as yf
import pandas
import tvscreener as tvs
from tvscreener import StockField, TimeInterval
from tvscreener.filter import FilterOperator
#query='BN'
def get_stock_info(query):
    """
    Performs gene set enrichment analysis using gseapy's enrichr function.
    Returns the results as a DataFrame or a detailed error message.

    Args:
        target_gl (list): List of gene symbols for enrichment analysis.

    Returns:
        # pd.DataFrame: DataFrame containing enrichment results or a single-row table with error details.
         dict: JSON containing enrichment results or error details.
    """

    try:
        screener = tvs.StockScreener()
        screener.search(query)
# Query a specific stock ticker
        ticker_info = screener.get()  # Example for Apple Inc.
# Display the results
        print(ticker_info)
        ticker_info.head(10)
        
    except Exception as e:
        # Log error details
        print("Error during query:", str(e))
        

        # Return error as a DataFrame
        error_df = pd.DataFrame({
            "Error": [str(e)],
            "Details": [
                f"Processed  list: {target_gl}. "
                
            ]
        })
        return error_df
# Create a StructuredTool for the function
stock_tool = Tool.from_function(
    name="get_stock_info",
    func=get_stock_info,
    description=(
        "This tool performs stock information using tvs. "
        #"Input: A list of gene symbols. Output: A DataFrame with results or error details."
    ),
    return_direct=True
)

In [3]:
### test search function from community tool
os.environ["TAVILY_API_KEY"]=Tav_API_KEY
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [4]:
#llm = OpenAI(model=' gpt-3.5-turbo-instruct', api_key= API_KEY,temperature=0)

In [14]:
TavilySearchResults

langchain_community.tools.tavily_search.tool.TavilySearchResults

In [23]:
llm = ChatOpenAI(temperature=0, model="gpt-4o",api_key= API_KEY,)
#search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
mytools2 = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events and real-time data"),
    Tool(
        name = "Stock",
        func=stock_tool.run,
        description="useful for when you need to answer questions about stock ticker"),
    
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to calculate the number of new cases each year based on the incidence rate in a population of 5000 individuals"
    )
]
agent2 = initialize_agent(mytools2, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [26]:
agent2.run('search the most active stock ticker and use stock_tool funrion to query ticker MMM')



> Entering new AgentExecutor chain...
To find the most active stock ticker, I would typically need access to real-time stock market data, which I don't have. However, I can proceed with querying the stock ticker "MMM" using the Stock tool function.

Action: Stock
Action Input: "MMM"        Symbol   Name                           Description  All Time High  \
0     NYSE:MMM    MMM                            3M Company     217.187203   
1    OTC:MMMPF  MMMPF  Mermaid Maritime Public Company Ltd.       0.180000   
2  NASDAQ:QMMM   QMMM                 QMMM Holdings Limited      13.000000   
3     OTC:MMMW   MMMW       Mass Megawatts Wind Power, Inc.    1200.000000   
4     OTC:MMMM   MMMM                 Quad M Solutions Inc.     110.000000   

   All Time Low  All Time Performance  Aroon Down (14)  Aroon Up (14)  \
0      2.148974           3550.108789        35.714286     100.000000   
1      0.150000             20.000000              NaN            NaN   
2      0.540000            

'Unable to retrieve current stock information for ticker "MMM". Please check a financial news website or stock market platform for the latest details.'

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-4o",api_key= API_KEY,)
#search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
mytools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events and real-time data"),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to calculate the number of new cases each year based on the incidence rate in a population of 5000 individuals"
    )
]
agent = initialize_agent(mytools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [21]:
agent.agent.llm_chain.prompt.template

"Answer the following questions as best you can. You have access to the following tools:\n\nSearch(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[list[str]]' = None, metadata: 'Optional[dict[str, Any]]' = None, run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config: 'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any') -> 'Any' - useful for when you need to answer questions about current events and real-time data\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - useful for when you need to calculate the number of new cases each year based on the incidence rate in

In [12]:
agent.run('what is the incidence rate of cancer in the US? What is the number of expected new cancer cases a year based on that?')



> Entering new AgentExecutor chain...
To answer this question, I need to find the current incidence rate of cancer in the US. Once I have that information, I can calculate the expected number of new cancer cases per year in a population of 5000 individuals.

Action: Search
Action Input: "current incidence rate of cancer in the US 2023"
Observation: [{'url': 'https://acsjournals.onlinelibrary.wiley.com/doi/full/10.3322/caac.21763', 'content': 'Jan 12, 2023 ... In 2023, 1,958,310 new cancer cases and 609,820 cancer deaths are projected to occur in the United States. Cancer incidence increased for'}, {'url': 'https://pubmed.ncbi.nlm.nih.gov/36633525/', 'content': 'Save citation to file\nEmail citation\nAdd to Collections\nAdd to My Bibliography\nYour saved search\nCreate a file for external citation management software\nYour RSS Feed\nFull text links\nActions\nShare\nPage navigation\nCancer statistics, 2023\nAffiliation\nCancer statistics, 2023\nAuthors\nAffiliation\nAbstract\nEach year

'The incidence rate of cancer in the US is approximately 440.5 new cases per 100,000 people per year. Based on this rate, in a population of 5000 individuals, we can expect about 22 new cancer cases per year.'

In [13]:
agent.run('what is the cancer immune therapy responsive rate of cancer in lung cancer? What is the number of expected new cancer cases a year based on that?')



> Entering new AgentExecutor chain...
To answer the question, I need to find the cancer immune therapy response rate for lung cancer and then calculate the expected number of new cancer cases per year based on that response rate in a population of 5000 individuals.

Action: Search
Action Input: "cancer immune therapy response rate lung cancer 2023"
Observation: [{'url': 'https://www.mdanderson.org/newsroom/aacr-lung-cancer-outcomes-significantly-improved-immunotherapy-based-treatment-given-before-after-surgery.h00-159617856.html', 'content': 'Oct 23, 2023 ... ... response (pCR) rates compared to chemotherapy alone for patients with operable non-small cell lung cancer (NSCLC), according to results of a'}, {'url': 'https://onlinelibrary.wiley.com/doi/10.1002/cai2.55', 'content': 'Feb 24, 2023 ... The 5-year survival rate can reach 18%–20%. Combined dual immunotherapy and chemotherapy are mainly suitable for PD-L1-negative patients, and'}, {'url': 'https://ascopost.com/issues/december-1

'The cancer immune therapy response rate for lung cancer using PD-1/PD-L1 inhibitors is approximately 29.5%. Based on this response rate, the expected number of new cancer cases per year in a population of 5000 individuals is approximately 1475 cases.'

In [ ]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)